<a href="https://colab.research.google.com/github/vkrisvasan/GenAIAgent/blob/main/AgentEx2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Building AI Agents With Crew AI using Google Gemini, Groq, LLama3
#CrewAI is an open-source framework that helps you build your own autonomous AI agents
#Creating two Agents one doign the research job and another doing the content writer job

#Gitaccount: vkrisvasan Repo: GenAIAgent Code: AgentEx2.ipynb
#Simple Agent assist implemented by
  #1. creating a project in google vertexai https://console.cloud.google.com/vertex-ai [note the projectID]
  #2. create a serper api key https://serpapi.com/dashboard to scrape Google and other search engines from serper's fast, easy, and complete API

In [ ]:
pip install crewai langchain-google-genai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
import os
import getpass
credential_names = ["GOOGLE_API_KEY","SERPER_API_KEY"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GOOGLE_API_KEY··········
Provide your...SERPER_API_KEY··········


In [ ]:
modelToBeused = "gemini-1.5-flash"

In [ ]:
from crewai import Agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model=modelToBeused,verbose=True, temperature=0.5,google_api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
researcher= Agent(
role="Senior Researcher",
goal="""Uncover groundbreaking topics
in {topic}""",
backstory=("""Driven by curiosity, you are at the forefront of Innovation sharin knowledge and news that would change the world""")
,tools=[],llm=llm,allow_delegation=True,verbose=True,memory=True,)

In [ ]:

writer= Agent(
role="Writer",
goal="Narrate compelling tech stories about {topic}",
backstory=("""Your expertise lies in breaking down complex topics into simpler a digestable knowledge, you specialize in
captivativing audience attention with yo writing and create engaging narratives and stories""")
,tools=[],llm=llm,allow_delegation=False,verbose=True,memory=True)

In [ ]:
from crewai_tools import SerperDevTool
os.environ["SERPER_API_KEY"]= os.getenv("SERPER_API_KEY")
tool= SerperDevTool()

In [ ]:
from crewai import Task

research_task= Task(description=(
"Identify the next big trend in {topic}."
"Focus on identifying pros and cons and the overall narrative."
                          "Your final report should clearly articulate the key points,"
                                  "its market opportunities, and potential risks."
),
expected_output='A comprehensive 3 paragraphs long report on the latest AI trend',
tools=[tool],
agent=researcher)

write_task = Task(description=(
"Compose an insightful article on {topic}."
"Focus on the latest trends and how it's impacting the industry."
        "This article should be easy to understand, engaging, and positive."
),
expected_output='A 4 paragraph article on {topic} advancements formatted as markdown',
                  tools=[tool],
agent=writer,
async_execution=False,
output_file='new-blog-post.md' # output file containing final draft
)

In [ ]:
from crewai import Crew,Process

crew= Crew(agents=[researcher,writer],
           tasks=[research_task,write_task],process= Process.sequential )


In [ ]:
result= crew.kickoff(inputs={"topic":"AI in Retail"})

print(result)



> Entering new CrewAgentExecutor chain...
Thought: To identify the next big trend in AI in retail, I need to understand the current landscape and emerging technologies. 

Action: Search the internet
Action Input: { "search_query": "AI trends in retail 2023" } 
 

{'message': 'Unauthorized.', 'statusCode': 403}

Thought: I need to find a way to access information about AI trends in retail. Since the internet search is blocked, I'll need to find an alternative.

Action: Delegate work to coworker
Action Input: { "coworker": "Writer", "task": "Research the current trends in AI in retail, focusing on emerging technologies and their potential impact. Provide a summary of the key trends, including pros and cons, market opportunities, and potential risks.", "context": "I need information on the latest trends in AI for retail, but I'm unable to access the internet. Please research this topic and provide a summary of your findings. Focus on emerging technologies and their potential impact, inc